In [1]:
import cv2 
from PIL import Image 
import numpy as np 
import pandas as pd 
import os
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray

img_dir = 'train'
mask_dir = 'train/masks'
excel_file = 'train/classif.xlsx'

# Load images
def load_images(img_dir, count):
    images = []
    for i in range(1, count + 1):
        img_path = os.path.join(img_dir, f"{i}.jpg")
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB format
            images.append(img)
        else:
            print(f"Image {img_path} not found.")
    return images

# Load masks
def load_masks(mask_dir, count):
    masks = []
    for i in range(1, count + 1):
        mask_path = os.path.join(mask_dir, f"binary_{i}.tif")
        if os.path.exists(mask_path):
            mask = Image.open(mask_path)
            mask = np.array(mask)
            masks.append(mask)
        else:
            print(f"Mask {mask_path} not found.")
    return masks

# Load classification file
def load_classification(excel_file):
    if os.path.exists(excel_file):
        return pd.read_excel(excel_file)
    else:
        print(f"Excel file {excel_file} not found.")
        return None
    
# Print        
images = load_images(img_dir, 250)
masks = load_masks(mask_dir, 250)
classif_df = load_classification(excel_file)    

print(f"Loaded {len(images)} images and {len(masks)} masks.")
print(classif_df.head())



Loaded 250 images and 250 masks.
   ID bug type         species
0   1      Bee  Apis mellifera
1   2      Bee  Apis mellifera
2   3      Bee  Apis mellifera
3   4      Bee  Apis mellifera
4   5      Bee  Apis mellifera


In [2]:
import torch
import detectron2
from concurrent.futures import ThreadPoolExecutor
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo


# Load YOLOv5 model
def load_yolov5_model():
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

# Perform YOLOv5 detection
def yolo_detect(model, image):
    results = model(image)
    return results

# Extract YOLOv5 features
def extract_yolo_features(results):
    boxes = results.xyxy[0].cpu().numpy()[:, :4]  # Bounding boxes
    labels = results.xyxy[0].cpu().numpy()[:, -1]  # Class labels
    return boxes, labels

yolo_model = load_yolov5_model()

# Perform detection on images and extract features
yolo_features_list = []
for img in images:
    results = yolo_detect(yolo_model, img)
    boxes, labels = extract_yolo_features(results)
    for box, label in zip(boxes, labels):
        yolo_features_list.append({
            'xmin': box[0], 'ymin': box[1], 'xmax': box[2], 'ymax': box[3],
            'label': label
        })

yolo_features_df = pd.DataFrame(yolo_features_list)
print(yolo_features_df.head())


Using cache found in /Users/zelin/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-25 Python-3.12.2 torch-2.3.0 CPU



: 